In [1]:
# Imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

# spark = SparkSession.builder.appName('733').getOrCreate()

# # Using the integrated file to start working on
# integrated_df = spark.read.parquet('/user/vcs/annual_integrated_dataset_v2.parquet')

# # Using nullcounts to filter columns to keep
# nullcounts = integrated_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in integrated_df.columns])
# nc = list(nullcounts.first())

# # Extracting out an industrial segment and modelling on it instead of the whole dataset
# # Services-packaged software category selection (from EDA)
# services_prepacked_software = integrated_df.filter(integrated_df.sic == '7372')
# print('Total records in integrated file: ', integrated_df.count())
# print(services_prepacked_software.show())
# print('Number of records in Services-packaged software industrial category: ', services_prepacked_software.count())

# # Reusing preprocessing steps implemented by Vincent
# some_dict = {}
# for x in services_prepacked_software.columns:
# 	some_dict[x] = 0

# nwdf = services_prepacked_software.fillna(some_dict)

# good_columns = []
# for i in range(0, len(nc)):
# 	if nc[i] == 0:
# 		good_columns.append(i)

# great_columns = [nwdf.columns[i] for i in good_columns]
# great_columns.append('rea')
# nwdf = nwdf.fillna(some_dict)

# non_string_columns = [k for (k,v) in nwdf.dtypes if v != 'string']
# nwdf_no_strings = nwdf.select(*non_string_columns)
# feature_columns = [item for item in nwdf_no_strings.columns if item not in ['rea', 'features']]

# assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
# final_df = assembler.transform(nwdf_no_strings)
# final_final_df = final_df.drop(*feature_columns)

# final_final_df = final_final_df.withColumn('boolean_label', final_final_df.rea != 0)

# print('Class distribution: ', final_final_df.groupBy('boolean_label').count().show())

# final_final_df = final_final_df.withColumn('label', final_final_df.boolean_label.cast('float'))
# final_final_df = final_final_df.drop('rea').drop('boolean_label')
# print(final_final_df.show())

# # String indexing not required
# stringIndexer = StringIndexer(inputCol="label", outputCol="indexed")
# si_model = stringIndexer.fit(final_final_df)
# td = si_model.transform(final_final_df)

# # Evaluators
# evaluator = MulticlassClassificationEvaluator(metricName = 'accuracy')
# eval = BinaryClassificationEvaluator()

# # RandomForest classifier
# rf = RandomForestClassifier(numTrees=100, maxDepth=16, labelCol="indexed", seed=42)
# model = rf.fit(td)
# result = model.transform(final_final_df)
# print('Accuracy on training data: ', evaluator.evaluate(result))

# # Train test split for model evaluation
# train, test = final_final_df.randomSplit([0.7, 0.3], seed=12345)

# rf = RandomForestClassifier(numTrees=100, maxDepth=16, labelCol="label", seed=42)
# print('Training RandomForest model on training set. \n Model parameters: {}'.format(rf._paramMap))
# trained_model = rf.fit(train)
# res = trained_model.transform(test)
# metrics = MulticlassMetrics(res.select(['label', 'prediction']).rdd)
# print('Accuracy on test set: ', evaluator.evaluate(res))
# print('Precision on test data: ', metrics.precision())
# print('Recall on test data: ', metrics.recall())
# print('F1 Score on test data: ', metrics.fMeasure())
# print('Area under ROC curve: ', eval.evaluate(res))

# # Logistic regression
# print('Training LogisticRegression model on training set.')
# logistic = LogisticRegression(regParam=0.1, labelCol="label")
# trained_model = logistic.fit(train)
# res = trained_model.transform(test)
# metrics = MulticlassMetrics(res.select(['label', 'prediction']).rdd)
# print('Accuracy on test set: ', evaluator.evaluate(res))
# print('Precision on test data: ', metrics.precision())
# print('Recall on test data: ', metrics.recall())
# print('F1 Score on test data: ', metrics.fMeasure())
# print('Area under ROC curve: ', eval.evaluate(res))

In [2]:
spark = SparkSession.builder.appName('733').getOrCreate()

In [11]:
# Using the integrated file to start working on
integrated_df = spark.read.csv('/home/kanika/accounting-ml-project/AIprojectcopy/annual_compustat.csv',header=True,inferSchema=True)

In [15]:
ls = ['CONM', 'ACCHG', 'ACO', 'ACOMINC', 'ACQCSHI', 'ACQGDWL', 'ACQINTAN', 'ACQLNTAL', 'ACT', 'AMGW','AQ','AP','AQC','ARC','AT','AU','AUOP','AUOPIC','BKVLPS','CAPX','CDVC','CEQ','CGA','CHE','CHECH','CI','CIMII','CLD2','CLD3','CLD4','CLD5','CLT','COGS','CSHI','CURNCD','DCOM','DCPSTK','DCVT','DEPC','DLTT','DM','DO','DP','DPACLS','DRC','DRLT','DV','DVC','DVPA','DVPDP','DVPSP_F','EBIT','EBITDA','EMOL','EMP','EPSFI','EPSPI','ESOPCT','ESOPDLT','EXCHG','EXRE','FATB','FATE','FATN','FATL','FATO','FCA','FEL','FIC','FINCF','FOPT','FYEAR','FYR','GDWL','GDWLAM','GDWLIA','GLA','GP','GWO','HEDGEGL','ICAPT','IDBFLAG','IDIIS','IDILC','IDIT','INTAN','INTC','INVCH','INCFG','INVOFS','INVRM','INVT','INVWIP','IPODATE','ISEQ','ISEQC','ISEQM','ISFI','ISGR','ISGU','ITCB','IVCH','IVNCF','LCT','LCUACU','LIFR','LLRCI','LLWOCI','LT','MIB','MIBT','MII','MKVALT','NAICS','NI','NIINT','NIINTPFC','NIINTPFP','NOPIO','NP','NPANL','NPAT','NRTXT','OANCF','OB','OIADP','OIBDP','OPEPS','OPINI','OPITI','OPREPSX','OPTCA','OPTEX','OPTEXD','OPTFVGR','OPTGR','OPTPRCCA','OPTPRCEX','OPTPRCEY','OPTPRCGR','OPTPRCWA','OPTVOL','PCL','PI','PIFO','PLL','PNCA','PNCIA','PPEGT','PPENB','PPENC','PPENLS','PPENNR','PPENT','PPEVO','PPEVR','PRCC_F','PRCH_F','PRICAN','PRIROW','PRIUSA','PRODV','PRSTKCC','PRSTKPC','PSTKC','PVCL','PVO','PVPL','PVT','RANK','RCA','RDIP','RE','REA','REAJO','RECCH','RECT','RECTA','REUNA','REVT','RLL','RMUM','RPAG','RVLRV','RVTI','SALE','SALEPFC','SALEPFP','SCO','SCSTKC','SEQ','SIC','SIV','SPCE','SPCINDCD','SPCSECCD','SPCSRC','SPI','SPPE','SPPIV','SPSTKC','SRET','SSNP','STBO','STIO','STKCO','STKO','TDC','TEQ','TFVA','TFVCE','TFVL','TIC','TLCF','TRANSA','TSA','TSTKC','TSTKP','TXC','TXDB','TXDBA','TXDBCA','TXDBCL','TXFED','TXFO','TXPD','TXR','TXT','TXTUBXINTBS','TXTUBXINTIS','VPAC','VPO','WCAP','WCAPC','WCAPCH','WDA','XACC','XAD','XAGO','XAGT','XCOM','XDP','XEQO','XI','XIDO','XINST','XINT','XINTD','XINTOPT','XLR','XOPR','XOPRAR','XPP','XPR','XRD','XRENT','XS','XSGA','XSTF','XSTFO','XSTFWS','XT']
ls = [x.lower() for x in ls]
annual_filtered = integrated_df.select([c for c in integrated_df.columns if c in ls])

In [20]:
# Using nullcounts to filter columns to keep
nullcounts = annual_filtered.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in annual_filtered.columns])
nc = list(nullcounts.first())
# Extracting out an industrial segment and modelling on it instead of the whole dataset

# Services-packaged software category selection (from EDA)
services_prepacked_software = annual_filtered.filter(annual_filtered.sic == '7372')
print('Total records in integrated file: ', annual_filtered.count())
print(services_prepacked_software.show())
print('Number of records in Services-packaged software industrial category: ', services_prepacked_software.count())
# Reusing preprocessing steps implemented by Vincent
some_dict = {}
for x in services_prepacked_software.columns:
    some_dict[x] = 0
nwdf = services_prepacked_software.fillna(some_dict)
good_columns = []
for i in range(0, len(nc)):
    if nc[i] == 0:
        good_columns.append(i)
great_columns = [nwdf.columns[i] for i in good_columns]
great_columns.append('rea')
nwdf = nwdf.fillna(some_dict)
non_string_columns = [k for (k,v) in nwdf.dtypes if v != 'string']
nwdf_no_strings = nwdf.select(*non_string_columns)
feature_columns = [item for item in nwdf_no_strings.columns if item not in ['rea', 'features']]

Total records in integrated file:  505781
+-----+------+----------------+------+---+-----+-----+-------+-------+-------+--------+--------+------+----+-----+------+----+------+-------+-----+----+-------+----+------+-------+----+-----+----+----+----+----+----+------+----+----+------+----+----+-----+-----+----+-----+------+----+----+----+----+----+-----+-------+-------+----+-----+-----+-----+------+-------+------+----+-----+----+----+----+----+----+------+-------+------+------+------+----+-------+----+-------+-------+-----+-----+-----+------+----+------+------+------+------+------+----+-----+-----+----+----+----+----+------+------+------+------+----+-----+------+------+----+----+----+-------+-----+--------+--------+-----+-----+-----+----+-----+------+-----+-------+-------+-----+-----+-----+-------+-----+-----+------+-------+-----+--------+--------+--------+--------+--------+------+----+-------+------+----+----+-----+-----+-----+-----+------+------+-----+-----+-----+-----+-------+-------+-

Number of records in Services-packaged software industrial category:  11543


In [21]:
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
final_df = assembler.transform(nwdf_no_strings)
final_final_df = final_df.drop(*feature_columns)
final_final_df = final_final_df.withColumn('boolean_label', final_final_df.rea != 0)
print('Class distribution: ', final_final_df.groupBy('boolean_label').count().show())
final_final_df = final_final_df.withColumn('label', final_final_df.boolean_label.cast('float'))
final_final_df = final_final_df.drop('rea').drop('boolean_label')
print(final_final_df.show())

+-------------+-----+
|boolean_label|count|
+-------------+-----+
|         true|  715|
|        false|10828|
+-------------+-----+

Class distribution:  None
+--------------------+-----+
|            features|label|
+--------------------+-----+
|(246,[0,1,235,244...|  0.0|
|(246,[0,1,235,244...|  0.0|
|(246,[0,1,235,244...|  0.0|
|(246,[0,1,3,9,11,...|  0.0|
|(246,[0,1,3,9,11,...|  0.0|
|(246,[0,1,3,9,11,...|  0.0|
|(246,[0,1,3,9,11,...|  0.0|
|(246,[0,1,3,9,11,...|  0.0|
|(246,[0,1,3,9,11,...|  0.0|
|(246,[0,1,3,9,11,...|  0.0|
|(246,[0,1,3,9,11,...|  0.0|
|(246,[0,1,3,9,11,...|  0.0|
|(246,[0,1,3,9,11,...|  0.0|
|(246,[0,1,3,9,11,...|  0.0|
|(246,[0,1,3,9,11,...|  0.0|
|(246,[0,1,3,9,11,...|  0.0|
|(246,[0,1,3,9,11,...|  0.0|
|(246,[0,1,3,9,11,...|  0.0|
|(246,[0,1,3,9,11,...|  0.0|
|(246,[0,1,3,9,11,...|  0.0|
+--------------------+-----+
only showing top 20 rows

None


In [22]:
# Evaluators
evaluator = MulticlassClassificationEvaluator(metricName = 'accuracy')
eval = BinaryClassificationEvaluator()

In [27]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

In [37]:
def balanceweights(df):
    minoritynum=df.filter(df['label']==1).count()
    datasetcount=df.count()
    balancingratio=(datasetcount-minoritynum)/datasetcount
    def weights(k):
        if(k==1):
            minweights=1.0*balancingratio
            return minweights
        else:
            majorityweigths = (1.0*(1-balancingratio))
            return majorityweigths
    awesome = udf(weights, FloatType())
    df = df.withColumn('weights', awesome(df['label']))
    print(df.show())
    return df
    
new_final_df = balanceweights(final_final_df)
    
    

+--------------------+-----+---------+
|            features|label|  weights|
+--------------------+-----+---------+
|(246,[0,1,235,244...|  0.0|0.0619423|
|(246,[0,1,235,244...|  0.0|0.0619423|
|(246,[0,1,235,244...|  0.0|0.0619423|
|(246,[0,1,3,9,11,...|  0.0|0.0619423|
|(246,[0,1,3,9,11,...|  0.0|0.0619423|
|(246,[0,1,3,9,11,...|  0.0|0.0619423|
|(246,[0,1,3,9,11,...|  0.0|0.0619423|
|(246,[0,1,3,9,11,...|  0.0|0.0619423|
|(246,[0,1,3,9,11,...|  0.0|0.0619423|
|(246,[0,1,3,9,11,...|  0.0|0.0619423|
|(246,[0,1,3,9,11,...|  0.0|0.0619423|
|(246,[0,1,3,9,11,...|  0.0|0.0619423|
|(246,[0,1,3,9,11,...|  0.0|0.0619423|
|(246,[0,1,3,9,11,...|  0.0|0.0619423|
|(246,[0,1,3,9,11,...|  0.0|0.0619423|
|(246,[0,1,3,9,11,...|  0.0|0.0619423|
|(246,[0,1,3,9,11,...|  0.0|0.0619423|
|(246,[0,1,3,9,11,...|  0.0|0.0619423|
|(246,[0,1,3,9,11,...|  0.0|0.0619423|
|(246,[0,1,3,9,11,...|  0.0|0.0619423|
+--------------------+-----+---------+
only showing top 20 rows

None


In [39]:
train.show()

+--------------------+-----+---------+
|            features|label|  weights|
+--------------------+-----+---------+
|(246,[0,1,2,3,4,9...|  0.0|0.0619423|
|(246,[0,1,2,3,9,1...|  0.0|0.0619423|
|(246,[0,1,2,3,9,1...|  0.0|0.0619423|
|(246,[0,1,3,4,6,7...|  0.0|0.0619423|
|(246,[0,1,3,4,6,7...|  0.0|0.0619423|
|(246,[0,1,3,4,9,1...|  0.0|0.0619423|
|(246,[0,1,3,4,9,1...|  0.0|0.0619423|
|(246,[0,1,3,4,9,1...|  0.0|0.0619423|
|(246,[0,1,3,4,9,1...|  0.0|0.0619423|
|(246,[0,1,3,4,9,1...|  0.0|0.0619423|
|(246,[0,1,3,4,9,1...|  0.0|0.0619423|
|(246,[0,1,3,4,9,1...|  0.0|0.0619423|
|(246,[0,1,3,4,9,1...|  0.0|0.0619423|
|(246,[0,1,3,4,9,1...|  1.0|0.9380577|
|(246,[0,1,3,4,9,1...|  0.0|0.0619423|
|(246,[0,1,3,4,9,1...|  0.0|0.0619423|
|(246,[0,1,3,4,9,1...|  1.0|0.9380577|
|(246,[0,1,3,4,9,1...|  0.0|0.0619423|
|(246,[0,1,3,4,9,1...|  0.0|0.0619423|
|(246,[0,1,3,4,9,1...|  0.0|0.0619423|
+--------------------+-----+---------+
only showing top 20 rows



In [38]:
train, test = new_final_df.randomSplit([0.7, 0.3], seed=12345)

In [33]:
logistic = LogisticRegression(regParam=0.1, labelCol="label", weightCol='weights')

In [40]:
trained_model = logistic.fit(train)
res = trained_model.transform(test)
metrics = MulticlassMetrics(res.select(['label', 'prediction']).rdd)

In [41]:
print('Accuracy on test set: ', evaluator.evaluate(res))
print('Precision on test data: ', metrics.precision())
print('Recall on test data: ', metrics.recall())
print('F1 Score on test data: ', metrics.fMeasure())
print('Area under ROC curve: ', eval.evaluate(res))

Accuracy on test set:  0.7118985883030827
Precision on test data:  0.7118985883030827
Recall on test data:  0.7118985883030827
F1 Score on test data:  0.7118985883030827
Area under ROC curve:  0.7443683867547333
